# MEIRL

In [1]:
import pandas as pd
import numpy as np
import random

In [2]:
area = "melbourne"
filename = "data/" + area + "/adjacency_matrix.csv"
adjacency_matrix = pd.read_csv(filename, index_col=0)

In [3]:
expert_feature_count = adjacency_matrix.copy()
expert_feature_count["value"] = 0
expert_feature_count = expert_feature_count["value"]
expert_feature_count = expert_feature_count.to_frame()
learner_feature_count = expert_feature_count.copy()
test_feature_count = expert_feature_count.copy()

In [4]:
# cross_validation
all_data = 400
training_number = int(all_data * 0.75)
test_number = int(all_data * 0.25)

# Import expert trajectory

### prepare dataset for training

In [5]:
for i in range (1,training_number):
    filename = "data/" + area + "/" + str(i)
    file = open(filename, "r")
    expert_trajectory = file.read().split(', ')
    for j in expert_trajectory:
        current_count = int(expert_feature_count.loc[int(j), ["value"]].values[0])
        add_count = expert_trajectory.count(j)
        new_count = current_count + add_count 
        expert_feature_count.loc[int(j), ["value"]] = new_count
expert_feature_count = (expert_feature_count - expert_feature_count.min()) / expert_feature_count.max() - expert_feature_count.min()    

### prepare for testing

In [6]:
for i in range (training_number + 1 , all_data):
    filename = "data/" + area + "/" + str(i)
    file = open(filename, "r")
    expert_trajectory = file.read().split(', ')
    for j in expert_trajectory:
        current_count = int(test_feature_count.loc[int(j), ["value"]].values[0])
        add_count = expert_trajectory.count(j)
        new_count = current_count + add_count 
        test_feature_count.loc[int(j), ["value"]] = new_count
test_feature_count = (test_feature_count - test_feature_count.min()) / test_feature_count.max() - test_feature_count.min()    

# Let's train

In [12]:
#Initialization of reward parameter
reward_parameter = expert_feature_count.copy()
reward_parameter.columns = ["value"]
reward_parameter["value"] = np.random.random_sample(reward_parameter.shape[0])

In [13]:
def update_position(current_position, eps, reward):
    action_list = np.array(adjacency_matrix[adjacency_matrix[current_position] == 1].index)
    value_list = reward[reward.index.isin(action_list)]

    if np.random.random() > eps:
        updated_position = random.choice(value_list[value_list["value"] == max(value_list["value"])].index.tolist())
#         gained_reward = value_list.loc[updated_position, "value"]
    else:
        updated_position = random.choice(value_list["value"].index.tolist())
#         gained_reward = value_list.loc[updated_position, "value"]

    return updated_position

In [14]:
# how many iterations?
Y = []
iteration = 50
for i in range (iteration):
    epsilon = 1 / (i+1)  
    learner_feature_count["value"] = 0
    for j in range (1,training_number):
        start_position = random.choice(adjacency_matrix.columns)
        current_position = start_position
        # agent will walk for 50 steps
        for k in range(50):
            current_position = str(update_position(current_position, epsilon, reward_parameter))
            learner_feature_count.loc[int(current_position), ["value"]] += 1
    
    learner_feature_count = (learner_feature_count - learner_feature_count.min()) / (learner_feature_count.max() - learner_feature_count.min() )   
    gradient = (expert_feature_count - learner_feature_count) 
    reward_parameter = (gradient*0.2) + reward_parameter
    reward_parameter = (reward_parameter - reward_parameter.min()) / (reward_parameter.max() - reward_parameter.min() )   
    
    difference = gradient.abs().sum().values[0]
    Y.append(difference)
    print(i, difference)
    
    correlation_dataframe = pd.concat([learner_feature_count, test_feature_count], axis=1)
    correlation = correlation_dataframe.corr().values[0][1]
    print(correlation)

0 166.854655712
0.164724495848
1 151.466199607
0.0992081310543
2 135.630363985
0.122726958931
3 126.085401891
0.214375016211
4 126.713574423
0.165832720479
5 121.036246612
0.212229163469
6 116.471846847
0.298192285708
7 116.076550388
0.324494377031
8 116.729310742
0.334857763278
9 121.239355231
0.277610341971
10 113.230777224
0.34386684914
11 116.969755971
0.290052511019
12 116.13145682
0.306686045823
13 116.386987091
0.286053385466
14 117.838963964
0.2955491961
15 120.120614035
0.239570650549
16 118.711839266
0.307171092327
17 117.195378675
0.291053510441
18 114.930445762
0.345883314093
19 117.643262738
0.304357306731
20 119.706604426
0.254291146704
21 118.157563256
0.309879975049
22 116.290057915
0.318720888177
23 116.08247296
0.316734164838
24 118.276225243
0.283470063041
25 120.382077141
0.302432117478
26 118.279352806
0.327451564845
27 121.852486426
0.291304372539
28 120.869232735
0.26873277898
29 118.2723466
0.328971361442
30 119.7111678
0.2963688247
31 118.462536298
0.3382485284

# Local Depth

In [7]:
#local depth
filename = "data/" + area + "/" + "closeness_centrality.csv"

closeness_centrality = pd.read_csv(filename, index_col=0)
adjacent_node = adjacency_matrix.sum().to_frame()
adjacent_node.columns = ["closeness_centrality"]
adjacent_node.index = adjacent_node.index.map(int)
mean_depth = (closeness_centrality - adjacent_node) / (adjacent_node.shape[0] - 1)
relative_asymmetry = (2*(mean_depth - 1)) / (adjacent_node.shape[0] - 2)
relative_asymmetry = (relative_asymmetry - relative_asymmetry.min()) / (relative_asymmetry.max() - relative_asymmetry.min() )   

In [8]:
correlation_dataframe = pd.concat([relative_asymmetry, test_feature_count], axis=1)
correlation_local_depth = - correlation_dataframe.corr().values[0][1]

In [9]:
correlation_local_depth

0.44008136920756391

# Connectivity 

In [10]:
mean_depth = adjacent_node / (adjacent_node.shape[0] - 1)
relative_asymmetry = (2*(mean_depth - 1)) / (adjacent_node.shape[0] - 2)
relative_asymmetry = (relative_asymmetry - relative_asymmetry.min()) / (relative_asymmetry.max() - relative_asymmetry.min() )   
correlation_dataframe = pd.concat([relative_asymmetry, test_feature_count], axis=1)
correlation_connectivity = correlation_dataframe.corr().values[0][1]
correlation_connectivity

0.33203235274366627

# Global Depth

In [11]:
mean_depth = closeness_centrality / (adjacent_node.shape[0] - 1)
relative_asymmetry = (2*(mean_depth - 1)) / (adjacent_node.shape[0] - 2)
relative_asymmetry = (relative_asymmetry - relative_asymmetry.min()) / (relative_asymmetry.max() - relative_asymmetry.min() )   
correlation_dataframe = pd.concat([relative_asymmetry, test_feature_count], axis=1)
correlation_global_depth = - correlation_dataframe.corr().values[0][1]
correlation_global_depth

0.4400805919569265